# ETL

## Extract

Zuerst wollen wir die Daten herunterladen. Hier ist zu beachten, dass dieser Datensatz Linzensiert ist, weshabl er hier auch nicht direkt mit aufgenommen wurde. Der Datensatz kann einerseits [von IEEE](https://ieee-dataport.org/documents/realwaste#files) heruntergeladen werden, dafür wird jedoch ein IEEE Account benötigt. Alternativ kann er auch mit der folgenden Zelle von GitHub heruntergeladen werden. 

In [1]:
# Download the files from GitHub
# This dataset is licensed under a Creative Commons Attribution 4.0 International (CC BY 4.0) license.
!git clone https://github.com/sam-single/realwaste.git



Cloning into 'realwaste'...
remote: Enumerating objects: 4779, done.
remote: Counting objects: 100% (15/15), done.
remote: Compressing objects: 100% (10/10), done.
remote: Total 4779 (delta 3), reused 8 (delta 1), pack-reused 4764
Receiving objects: 100% (4779/4779), 655.91 MiB | 10.21 MiB/s, done.
Resolving deltas: 100% (3/3), done.
Updating files: 100% (4753/4753), done.


In [ ]:
import os

# get path of current file
